In [ ]:
from newspaper import Article
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import string
import nltk
from nltk.stem import WordNetLemmatizer
import warnings
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('punkt', quiet=True)

In [ ]:
article = Article('https://www.who.int/news-room/fact-sheets/detail/middle-east-respiratory-syndrome-coronavirus-(mers-cov)')
article.download()
article.parse()
article.nlp()
mers = article.text

In [ ]:
text = mers
list_kalimat = nltk.sent_tokenize(text)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
intents = json.loads(open('intents.json').read())

In [ ]:
words  = pickle.load(open('word.pkl', 'rb'))
classes  = pickle.load(open('classes.pkl', 'rb'))
model  = load_model('ChatBot.model')

In [ ]:
def clean_sent(sentence):
    sent_words = nltk.word_tokenize(sentence)
    sent_words = [lemmatizer.lemmatize(word) for word in sent_words]
    return sent_words

In [ ]:
def bag_words(sentence):
    sent_words = clean_sent(sentence)
    bag = [0] * len(words)
    for w in sent_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    bow = bag_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERR_TRES = 0.1
    results = [[i, r] for i, r in enumerate(res) if r > ERR_TRES]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [ ]:
def conv_bot(intents_list, intents_json):
    tag = intents_list[0]['intent']
    loi = intents_json['intents']
    for i in loi:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def sapa_bot(input_user):
    bot_greet = ["Hello", "Hy", "Hola", "Wellcome", "Shalom", "Bonjour"]
    user_greet = ["Hello", "Hy", "Hi", "Hola"]

    for word in input_user.split():
        if word in user_greet:
            return random.choice(bot_greet)

In [ ]:
def sort_index(var_list):
    length = len(var_list)
    list_index = list(range(0, length))

    x = var_list
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    return list_index

In [ ]:
def balasan_bot(user_input):
    list_kalimat.append(user_input)
    bot_resp = ''
    cm = CountVectorizer().fit_transform(list_kalimat)
    nilai_kesamaan = cosine_similarity(cm[-1], cm)
    list_nilai_kesamaan = nilai_kesamaan.flatten()
    index = sort_index(list_nilai_kesamaan)
    index = index[1:]
    response_flags = 0

    f = 0
    for i in range(len(index)):
        if list_nilai_kesamaan[index[i]] > 0.0:
            bot_resp = bot_resp + " " + list_kalimat[index[i]]
            response_flags = 1
            f += 1
        if f > 2:
            break

    if response_flags == 0:
        bot_resp = bot_resp + " " + "Sorry, your question is invalid or the answer is unavailable"
    list_kalimat.remove(user_input)

    return bot_resp

In [ ]:
print("Wellcome to the UN WHO Site ABout MERS-CoV")

exit_sent = ["Keluar", "Quit", "Exit", "Bye"]

while(True):
    input_user = input().capitalize()
    print("User:", input_user)

    if input_user in exit_sent:
        print("Thank you for coming. See you soon!")
        break
    else:
        if sapa_bot(input_user) != None:
            print("WHO Bot:", sapa_bot(input_user))
        else:
            print("WHO Bot:", balasan_bot(input_user))